In [1]:
import findspark
findspark.init('/opt/spark')

from pyspark import SparkConf
from pyspark.sql import SparkSession


In [2]:
# Init variables

SPARK_URI = "spark://10.10.10.80:7077"
SPARK_APP = "Extract hostnames of Telnet"

spark = SparkSession.builder \
    .master(SPARK_URI) \
    .appName(SPARK_APP) \
    .config("spark.driver.host", "10.10.10.80") \
    .config("spark.jars", "postgresql-42.2.20.jar") \
    .getOrCreate()



In [3]:
PG_URL="jdbc:postgresql://localhost:5432/honeypot"
PG_USER="max"
PG_PASS="K8yrQKMD151"

options = {
    "url" : PG_URL,
    "user" : PG_USER,
    "password" : PG_PASS,
    "driver" : "org.postgresql.Driver"
}

df_log = spark.read.format("jdbc").options(
    dbtable="log",
    **options
).load()


In [4]:

# from datetime import date
from dateutil import parser

dates = [
    "2021-05-17",
    "2021-05-18",
    "2021-05-19",
    "2021-05-20",
    "2021-05-21",
    "2021-05-22",
    "2021-05-23",
    "2021-05-24",
    "2021-05-25",
    "2021-05-26",
    "2021-05-27",
    "2021-05-28",
    "2021-05-29",
    "2021-05-30",
    "2021-05-31",
]

# dates = [date.fromisoformat(d) for d in dates]
dates = [parser.parse(d) for d in dates]

df_telnet = {}

for i in range(len(dates) - 1):
    print(f"Load {dates[i]} to {dates[i+1]}")
    df_telnet[dates[i]] = df_log.select("timestamp", "raw").filter("category == 'telnet'").filter(f"timestamp >= '{dates[i]}' and timestamp < '{dates[i+1]}'")

#df_telnet[dates[0]].show(20);


Load 2021-05-17 00:00:00 to 2021-05-18 00:00:00
Load 2021-05-18 00:00:00 to 2021-05-19 00:00:00
Load 2021-05-19 00:00:00 to 2021-05-20 00:00:00
Load 2021-05-20 00:00:00 to 2021-05-21 00:00:00
Load 2021-05-21 00:00:00 to 2021-05-22 00:00:00
Load 2021-05-22 00:00:00 to 2021-05-23 00:00:00
Load 2021-05-23 00:00:00 to 2021-05-24 00:00:00
Load 2021-05-24 00:00:00 to 2021-05-25 00:00:00
Load 2021-05-25 00:00:00 to 2021-05-26 00:00:00
Load 2021-05-26 00:00:00 to 2021-05-27 00:00:00
Load 2021-05-27 00:00:00 to 2021-05-28 00:00:00
Load 2021-05-28 00:00:00 to 2021-05-29 00:00:00
Load 2021-05-29 00:00:00 to 2021-05-30 00:00:00
Load 2021-05-30 00:00:00 to 2021-05-31 00:00:00


In [5]:
from pyspark.sql import functions as fn
import ipaddress

path = "/home/max/jupyter/dataplane/export/"
#df.select("raw").split("' from '")

def is_host(s):
    try:
        ipaddress.ip_address(s)
        return False
    except:
        return True

def df_to_list(df):
    df = df.select(fn.split("raw", "' from '")[1].alias("raw"))
    df = df.select(fn.split("raw", "' FOR '")[0].alias('host'))
    df = df.distinct()
    arr = df.collect()
    arr = [r.host for r in arr]
    arr = [x for x in arr if is_host(x)]
    return arr

#tmp = df.select(fn.split("raw", "' from '")[1].alias('raw'))
#tmp = tmp.select(fn.split("raw", "' FOR '")[0].alias('host'))
#tmp = tmp.distinct()
#tmp = tmp.select(fn.map_filter("host", is_ip).alias("host"))
#tmp = tmp.select(
#    fn.filter("host", is_ip).alias("host")
#)
#tmp = tmp.filter(tmp.host)

#tmp = tmp.collect()
#tmp = [r.host for r in tmp]
#tmp = list(filter(is_host, tmp))

#print(tmp)

hosts = {}

for x in dates:
    print(f"Dataframe to host list {x}")
    if x in df_telnet.keys():
        hosts[x] = df_to_list(df_telnet[x])


for k, v in hosts.items():
    _f = f"{path}/hosts-{k.year}-{k.month}-{k.day}.txt"
    
    print(f"Write {len(v)} hosts to file {_f}")
    with open(_f, 'w') as f:
        f.writelines([x + '\n' for x in v])

Dataframe to host list 2021-05-17 00:00:00
Dataframe to host list 2021-05-18 00:00:00
Dataframe to host list 2021-05-19 00:00:00
Dataframe to host list 2021-05-20 00:00:00
Dataframe to host list 2021-05-21 00:00:00
Dataframe to host list 2021-05-22 00:00:00
Dataframe to host list 2021-05-23 00:00:00
Dataframe to host list 2021-05-24 00:00:00
Dataframe to host list 2021-05-25 00:00:00
Dataframe to host list 2021-05-26 00:00:00
Dataframe to host list 2021-05-27 00:00:00
Dataframe to host list 2021-05-28 00:00:00
Dataframe to host list 2021-05-29 00:00:00
Dataframe to host list 2021-05-30 00:00:00
Dataframe to host list 2021-05-31 00:00:00
Write 41 hosts to file /home/max/jupyter/dataplane/export//hosts-2021-5-17.txt
Write 62 hosts to file /home/max/jupyter/dataplane/export//hosts-2021-5-18.txt
Write 70 hosts to file /home/max/jupyter/dataplane/export//hosts-2021-5-19.txt
Write 57 hosts to file /home/max/jupyter/dataplane/export//hosts-2021-5-20.txt
Write 64 hosts to file /home/max/jupyte